In [1]:
from helpers.utilities import *
%run helpers/notebook_setup.ipynb

In [2]:
rna_deseq2_path = 'data/clean/rna/all_samples.csv'
rna_counts_path = 'data/clean/rna/all_samples_counts.csv'

clinical_path = 'data/clean/clinical/data_with_derived_variables.csv'

# output
out_deseq2_path = 'data/clean/rna/clinical_data_ordered_to_match_rna_deseq2.csv'
out_counts_path = 'data/clean/rna/clinical_data_ordered_to_match_rna_counts.csv'

In [3]:
clinical_data = read_csv(clinical_path, index_col=0)

rna_deseq2 = read_csv(rna_deseq2_path, index_col=[0, 1])
raw_counts = read_csv(rna_counts_path, index_col=[0])

In [4]:
rna_deseq2.head()

,,001.TMD,006.CM,012.BM,016.CM,017.TMD,...,174.CM,011.TMR,043.TMS,078.CM,261.CM
ensembl_id,ensembl_gene_id,,,,,,,,,,,
ENSG00000000003,TSPAN6,11.596119,7.806308,1.391555,39.347366,24.765008,...,4.431071,51.810470,35.628629,0.000000,20.705553
ENSG00000000005,TNMD,0.000000,0.000000,0.000000,2.459210,0.000000,...,0.000000,11.102244,0.000000,0.000000,4.141111
ENSG00000000419,DPM1,0.000000,31.225230,36.876198,0.000000,72.313823,...,25.478656,11.102244,0.000000,161.520987,39.754662
ENSG00000000457,SCYL3,127.557309,145.717740,86.276387,95.909204,123.825040,...,121.854440,96.219444,3.958737,92.395480,125.889763
ENSG00000000460,C1orf112,46.384476,0.000000,50.791744,51.643418,14.859005,...,33.233029,49.960096,38.267787,0.000000,30.644219


In [5]:
raw_counts.head()

,001.TMD,006.CM,012.BM,016.CM,017.TMD,...,158.TMD,167.TMR,175.TMD,233.CM,261.CM
ENSG00000000003,1.0,3.0,2.00,16.00,25.00,...,0.0,0.0,0.0,0.0,25.00
ENSG00000000005,0.0,0.0,0.00,1.00,0.00,...,0.0,0.0,0.0,0.0,5.00
ENSG00000000419,0.0,12.0,53.00,0.00,73.00,...,0.0,0.0,0.0,0.0,48.00
ENSG00000000457,11.0,56.0,123.84,38.72,124.95,...,0.0,0.0,0.0,0.0,151.94
ENSG00000000460,4.0,0.0,73.16,21.28,15.05,...,0.0,0.0,0.0,0.0,37.06


Note: the raw counts are not really raw counts. These are results of RSEM, which seemingly outputs floats not integers.

How to approach that?
- when using DESeq2 "tximport" function [was recommended](https://support.bioconductor.org/p/94003/#94028)
   - [tximport](https://bioconductor.org/packages/devel/bioc/vignettes/tximport/inst/doc/tximport.html) is a package on bioconductor and the docs describe how to use it for limma-voom
   - it has a related publication: https://f1000research.com/articles/4-1521/v1

Take only rows for patients with RNASeq data, ordered to match the corresponding data frames:

In [6]:
clinical_data_ordered_for_deseq2 = clinical_data.loc[rna_deseq2.columns]
clinical_data_ordered_for_deseq2.to_csv(out_deseq2_path)
clinical_data_ordered_for_deseq2.head()

,AdmissionDate,Birthday,Sex,PrevTB,PrevTBForm,...,survival,censored_survival,Meningitis,Tuberculosis,Meningitis_with_tuberculosis_status
001.TMD,2015-02-06,1980-01-04,M,False,NaN,...,NaN,182,Tuberculosis,Definite,Definite tuberculosis
006.CM,2015-02-11,1966-05-05,M,True,Pulmonary,...,NaN,182,Cryptococcal,-,Cryptococcal
012.BM,2015-02-19,1961-01-09,M,False,NaN,...,NaN,182,Bacterial,-,Bacterial
016.CM,2015-02-20,1958-11-11,M,True,Pulmonary,...,24.0,24,Cryptococcal,-,Cryptococcal
017.TMD,2015-02-22,1957-07-03,F,False,NaN,...,161.0,161,Tuberculosis,Definite,Definite tuberculosis


In [7]:
clinical_data_ordered_for_counts = clinical_data.loc[raw_counts.columns]
clinical_data_ordered_for_counts.to_csv(out_counts_path)
clinical_data_ordered_for_counts.head()

,AdmissionDate,Birthday,Sex,PrevTB,PrevTBForm,...,survival,censored_survival,Meningitis,Tuberculosis,Meningitis_with_tuberculosis_status
001.TMD,2015-02-06,1980-01-04,M,False,NaN,...,NaN,182,Tuberculosis,Definite,Definite tuberculosis
006.CM,2015-02-11,1966-05-05,M,True,Pulmonary,...,NaN,182,Cryptococcal,-,Cryptococcal
012.BM,2015-02-19,1961-01-09,M,False,NaN,...,NaN,182,Bacterial,-,Bacterial
016.CM,2015-02-20,1958-11-11,M,True,Pulmonary,...,24.0,24,Cryptococcal,-,Cryptococcal
017.TMD,2015-02-22,1957-07-03,F,False,NaN,...,161.0,161,Tuberculosis,Definite,Definite tuberculosis


Please note the difference in number of patients between raw counts and dseq2 processed data:

In [8]:
len(clinical_data_ordered_for_counts), len(clinical_data_ordered_for_deseq2)

(56, 49)